# Imports

In [ ]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
import spacy
import os
import zipfile
import json
import copy
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.corpora import Dictionary


np.random.seed(2018)
import nltk
nltk.download('wordnet')
nltk.download('popular')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("popular")
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]  

# Data Mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

data = pd.read_csv('../data/raw/AirBnB/Samples/reviews.csv')

# Looking at Listing.csv


In [ ]:
listing_data = pd.read_csv('../data/raw/AirBnB/Samples/listings.csv')
listing_data.shape

(8127, 75)

In [ ]:
#we want id, description, neighborhood_overview, host_about, neighbourhood, review_score_rating
listfinaldata = listing_data[['id','description','neighborhood_overview','review_scores_value']]
listfinaldata.head()
reviewdatafinal = data[['listing_id','comments']]
reviewdatafinal.head()
data.shape

(517200, 6)

In [ ]:
finaldf= pd.merge(listfinaldata, reviewdatafinal, left_on="id",right_on='listing_id')
above4stars = finaldf[finaldf['review_scores_value']>=4.0]
midstars = finaldf[finaldf['review_scores_value'].between(3.0,3.99) ]
below3stars = finaldf[finaldf['review_scores_value']<3.0]
below3stars.sample(10)
midstars.dropna(inplace=True)
above4stars.dropna(inplace=True)
below3stars.dropna(inplace=True)

#finaldf = finaldf.sample(frac=0.05)


<ipython-input-6-d5a7ea4b9818>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  midstars.dropna(inplace=True)
<ipython-input-6-d5a7ea4b9818>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  above4stars.dropna(inplace=True)
<ipython-input-6-d5a7ea4b9818>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  below3stars.dropna(inplace=True)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import string


#Description and Neighborhood Overview and Reviews

## The Functions to Clean/Do LDA Unigram

In [ ]:
def clean_corpus(corpus):
   
    corpus = [text for text in corpus if str(text) != 'nan' and str(text).strip() != '']
    corpus = [re.sub(r'<[^>]+>', ' ', text) for text in corpus]
    corpus = [re.sub(r'[^a-zA-Z]', ' ', text) for text in corpus]
    corpus = [text.lower() for text in corpus]
    stop_words = set(stopwords.words('english'))
    corpus = [[word for word in text.split() if word not in stop_words] for text in corpus]
    lemmatizer = WordNetLemmatizer()
    corpus = [[lemmatizer.lemmatize(word) for word in text] for text in corpus]
    corpus = [word for text in corpus for word in text]

    return corpus

def get_lda_topics(word_list, num_topics=2, num_words=10):
  
    dictionary = gensim.corpora.Dictionary([word_list])
    corpus = [dictionary.doc2bow(word_list)]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=dictionary, num_topics=num_topics,passes=10,alpha=1.0,eta='auto')
    topics = lda_model.show_topics(num_topics=num_topics, num_words=num_words)

    return topics

  

In [ ]:
listfinaldata
dnotop = listfinaldata[listfinaldata['review_scores_value']>=4.0]
dnomid = listfinaldata[listfinaldata['review_scores_value'].between(3.0,3.99) ]
dnolow = listfinaldata[listfinaldata['review_scores_value']<3.0]

reviewdf = finaldf[['review_scores_value','listing_id','comments']]
revtop = reviewdf[reviewdf['review_scores_value']>=4.0]
revmid = reviewdf[reviewdf['review_scores_value'].between(3.0,3.99) ]
revlow = reviewdf[reviewdf['review_scores_value']<3.0]



### D&N_O LOW


In [ ]:
dnolow_d_corpus = dnolow['description']
dnolow_d_corpus = clean_corpus(dnolow_d_corpus)
get_lda_topics(dnolow_d_corpus)


[(0,
  '0.013*"nashville" + 0.013*"bedroom" + 0.012*"home" + 0.012*"space" + 0.011*"room" + 0.010*"music" + 0.010*"kitchen" + 0.009*"area" + 0.008*"guest" + 0.008*"bed"'),
 (1,
  '0.002*"bedroom" + 0.002*"space" + 0.002*"nashville" + 0.002*"music" + 0.002*"room" + 0.002*"area" + 0.002*"home" + 0.002*"kitchen" + 0.002*"living" + 0.002*"guest"')]

In [ ]:
dnolow_no_corpus = dnolow['neighborhood_overview']
dnolow_no_corpus = clean_corpus(dnolow_no_corpus)
get_lda_topics(dnolow_no_corpus)

[(0,
  '0.004*"min" + 0.004*"mile" + 0.004*"nashville" + 0.004*"park" + 0.004*"museum" + 0.004*"state" + 0.004*"tennessee" + 0.004*"neighborhood" + 0.004*"music" + 0.004*"area"'),
 (1,
  '0.068*"min" + 0.037*"mile" + 0.026*"nashville" + 0.022*"park" + 0.020*"museum" + 0.018*"state" + 0.015*"music" + 0.015*"tennessee" + 0.013*"neighborhood" + 0.011*"area"')]

### D&N_O MID

In [ ]:
dnomid_d_corpus = dnomid['description']
dnomid_d_corpus = clean_corpus(dnomid_d_corpus)
get_lda_topics(dnomid_d_corpus)

[(0,
  '0.001*"space" + 0.001*"nashville" + 0.001*"kitchen" + 0.001*"bedroom" + 0.001*"guest" + 0.001*"living" + 0.001*"home" + 0.001*"room" + 0.001*"downtown" + 0.001*"music"'),
 (1,
  '0.016*"space" + 0.014*"nashville" + 0.011*"bedroom" + 0.010*"home" + 0.010*"kitchen" + 0.009*"downtown" + 0.008*"guest" + 0.008*"living" + 0.008*"area" + 0.008*"room"')]

In [ ]:
dnomid_no_corpus = dnomid['neighborhood_overview']
dnomid_no_corpus = clean_corpus(dnomid_no_corpus)
get_lda_topics(dnomid_no_corpus)

[(0,
  '0.031*"nashville" + 0.021*"min" + 0.018*"mile" + 0.010*"music" + 0.009*"museum" + 0.009*"neighborhood" + 0.008*"marathon" + 0.008*"area" + 0.008*"village" + 0.008*"downtown"'),
 (1,
  '0.002*"nashville" + 0.002*"min" + 0.002*"mile" + 0.001*"museum" + 0.001*"music" + 0.001*"downtown" + 0.001*"neighborhood" + 0.001*"home" + 0.001*"restaurant" + 0.001*"area"')]

### D&N_O TOP

In [ ]:
dnotop_d_corpus = dnotop['description']
dnotop_d_corpus = clean_corpus(dnotop_d_corpus)
get_lda_topics(dnotop_d_corpus)

[(0,
  '0.021*"nashville" + 0.018*"space" + 0.014*"home" + 0.012*"bed" + 0.012*"bedroom" + 0.009*"room" + 0.009*"downtown" + 0.008*"guest" + 0.008*"kitchen" + 0.007*"full"'),
 (1,
  '0.015*"space" + 0.014*"home" + 0.013*"bedroom" + 0.012*"room" + 0.011*"nashville" + 0.010*"downtown" + 0.009*"kitchen" + 0.009*"guest" + 0.009*"bed" + 0.008*"bathroom"')]

In [ ]:
dnotop_no_corpus = dnotop['neighborhood_overview']
dnotop_no_corpus = clean_corpus(dnotop_no_corpus)
get_lda_topics(dnotop_no_corpus)

[(0,
  '0.025*"nashville" + 0.014*"mile" + 0.013*"neighborhood" + 0.011*"downtown" + 0.011*"walk" + 0.010*"restaurant" + 0.010*"minute" + 0.009*"min" + 0.009*"broadway" + 0.008*"home"'),
 (1,
  '0.030*"nashville" + 0.020*"mile" + 0.016*"neighborhood" + 0.014*"downtown" + 0.014*"restaurant" + 0.013*"minute" + 0.012*"music" + 0.010*"min" + 0.010*"bar" + 0.010*"walk"')]

### Reviews

In [ ]:
revlow_corpus = revlow['comments']
revlow_corpus = clean_corpus(revlow_corpus)
get_lda_topics(revlow_corpus)

[(0,
  '0.001*"stay" + 0.001*"home" + 0.001*"bed" + 0.001*"airbnb" + 0.001*"one" + 0.001*"get" + 0.001*"room" + 0.001*"host" + 0.001*"place" + 0.001*"time"'),
 (1,
  '0.009*"host" + 0.009*"stay" + 0.007*"get" + 0.007*"bed" + 0.007*"place" + 0.007*"airbnb" + 0.007*"home" + 0.006*"hotel" + 0.006*"room" + 0.006*"one"')]

In [ ]:
revmid_corpus = revmid['comments']
revmid_corpus = clean_corpus(revmid_corpus)
get_lda_topics(revmid_corpus)

[(0,
  '0.016*"place" + 0.015*"stay" + 0.014*"great" + 0.010*"location" + 0.009*"would" + 0.009*"clean" + 0.008*"room" + 0.007*"nice" + 0.007*"get" + 0.006*"u"'),
 (1,
  '0.002*"stay" + 0.002*"great" + 0.002*"place" + 0.001*"location" + 0.001*"would" + 0.001*"room" + 0.001*"get" + 0.001*"nice" + 0.001*"night" + 0.001*"area"')]

In [ ]:
revtop_corpus = revtop['comments']
revtop_corpus = clean_corpus(revtop_corpus)
get_lda_topics(revtop_corpus)

[(0,
  '0.031*"great" + 0.028*"stay" + 0.025*"place" + 0.017*"nashville" + 0.016*"location" + 0.015*"would" + 0.014*"clean" + 0.010*"host" + 0.010*"everything" + 0.009*"home"'),
 (1,
  '0.033*"great" + 0.028*"place" + 0.021*"stay" + 0.018*"location" + 0.015*"clean" + 0.013*"nashville" + 0.012*"house" + 0.012*"perfect" + 0.011*"host" + 0.010*"would"')]

### Creating DF

In [ ]:
words = []
for topic, weights in get_lda_topics(dnolow_d_corpus):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))

df = pd.DataFrame(words, columns=['Low Rated Description'])

for topic, weights in get_lda_topics(dnomid_d_corpus):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))
df1 = pd.DataFrame(words, columns=['Mid Rated Description'])

for topic, weights in get_lda_topics(dnotop_d_corpus):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))
df2 = pd.DataFrame(words, columns=['Top Rated Description'])


df_combined_descriptions = pd.concat([df, df1,df2], axis=1)
#df_combined_descriptions.to_csv('DescriptionLDA',index=False)
df_combined_descriptions





,Low Rated Description,Mid Rated Description,Top Rated Description
0,room,room,room
1,area,area,area
2,nashville,nashville,nashville
3,home,home,home
4,bed,living,living
5,space,bed,bed
6,bedroom,space,space
7,kitchen,bedroom,bedroom
8,music,kitchen,kitchen
9,downtown,music,music


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
words = []
for topic, weights in get_lda_topics(dnolow_no_corpus):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))

df3 = pd.DataFrame(words, columns=['Low Rated Neighborhood Overview'])

for topic, weights in get_lda_topics(dnomid_no_corpus):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))
df4 = pd.DataFrame(words, columns=['Mid Rated Neighborhood Overview'])

for topic, weights in get_lda_topics(dnotop_no_corpus):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))
df5 = pd.DataFrame(words, columns=['Top Rated Neighborhood Overview'])


df_combined_no = pd.concat([df3, df4,df5], axis=1)
#df_combined_no.to_csv('NeighborhoodOverviewLDA',index=False)
df_combined_no

,Low Rated Neighborhood Overview,Mid Rated Neighborhood Overview,Top Rated Neighborhood Overview
0,neighborhood,neighborhood,min
1,min,min,museum
2,museum,museum,tennessee
3,area,area,home
4,nashville,nashville,downtown
5,tennessee,tennessee,marathon
6,state,state,bar
7,park,park,neighborhood
8,mile,mile,area
9,university,university,walk


In [ ]:
words = []
for topic, weights in get_lda_topics(revlow_corpus):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))

df6 = pd.DataFrame(words, columns=['Low Reviews'])

for topic, weights in get_lda_topics(revmid_corpus):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))
df7 = pd.DataFrame(words, columns=['Mid Reviews'])

for topic, weights in get_lda_topics(revtop_corpus):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))
df8 = pd.DataFrame(words, columns=['Top Reviews'])


df_combined_rev = pd.concat([df6, df7,df8], axis=1)
#df_combined_rev.to_csv('ReviewLDA',index=False)
df_combined_rev

,Low Reviews,Mid Reviews,Top Reviews
0,airbnb,airbnb,airbnb
1,room,room,room
2,stay,great,perfect
3,house,location,place
4,place,nice,home
5,home,stay,get
6,get,house,host
7,host,place,everything
8,hotel,home,location
9,one,get,stay


## Bigram Functions

In [ ]:
from nltk import word_tokenize 
from nltk.util import ngrams

def get_bigrams(text):
    text = re.sub('<[^<]+?>', '', text)
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    text = text.lower()
    bigrams = ['_'.join(gram) for gram in ngrams(text.split(), 2)]
    return bigrams


### D&N_O LOW

In [ ]:
dnolow_d_bigram = dnolow.copy()
dnolow_d_bigram.dropna(subset=['description'], inplace=True)
dnolow_d_all_bigrams = dnolow_d_bigram['description'].apply(get_bigrams).sum()


get_lda_topics(dnolow_d_all_bigrams)


[(0,
  '0.001*"king_bed" + 0.001*"1_king" + 0.001*"sofa_living" + 0.001*"1_queen" + 0.001*"stainless_steel" + 0.001*"living_room" + 0.001*"music_city." + 0.001*"minutes_away" + 0.001*"newly_renovated" + 0.001*"hayes_street"'),
 (1,
  '0.003*"king_bed" + 0.002*"1_king" + 0.002*"walking_track" + 0.002*"bedroom,_1" + 0.002*"country_music" + 0.002*"newly_renovated" + 0.002*"minutes_away" + 0.002*"music_city." + 0.002*"living_room" + 0.002*"stainless_steel"')]

In [ ]:
dnolow_no_bigram = dnolow.copy()
dnolow_no_bigram.dropna(subset=['neighborhood_overview'], inplace=True)
dnolow_no_all_bigrams = dnolow_no_bigram['neighborhood_overview'].apply(get_bigrams).sum()


get_lda_topics(dnolow_no_all_bigrams)

[(0,
  '0.009*"tennessee_state" + 0.008*"mins._nashville" + 0.008*"mins._tennessee" + 0.005*"park10_mins." + 0.005*"museum11_mins." + 0.004*"(6.4_miles)," + 0.004*"mall_state" + 0.004*"music_row" + 0.004*"capitol_mall" + 0.004*"bicentennial_capitol"'),
 (1,
  '0.002*"tennessee_state" + 0.002*"mins._tennessee" + 0.002*"mins._nashville" + 0.002*"museum11_mins." + 0.002*"hall_fame" + 0.002*"quiet_neighborhood" + 0.002*"bicentennial_capitol" + 0.002*"capitol_mall" + 0.002*"park10_mins." + 0.002*"music_row"')]

###D&N_O MID

In [ ]:
dnomid_d_bigram = dnomid.copy()
dnomid_d_bigram.dropna(subset=['description'], inplace=True)
dnomid_d_all_bigrams = dnomid_d_bigram['description'].apply(get_bigrams).sum()


get_lda_topics(dnomid_d_all_bigrams)


[(0,
  '0.000*"stainless_steel" + 0.000*"living_room" + 0.000*"music_city" + 0.000*"walking_distance" + 0.000*"living_area" + 0.000*"queen_sleeper" + 0.000*"sleeper_sofa" + 0.000*"smart_tv" + 0.000*"brand_new" + 0.000*"fully_equipped"'),
 (1,
  '0.002*"stainless_steel" + 0.002*"living_room" + 0.002*"living_area" + 0.001*"music_city" + 0.001*"walking_distance" + 0.001*"smart_tv" + 0.001*"sleeper_sofa" + 0.001*"queen_sleeper" + 0.001*"fully_equipped" + 0.001*"brand_new"')]

In [ ]:
dnomid_no_bigram = dnomid.copy()
dnomid_no_bigram.dropna(subset=['neighborhood_overview'], inplace=True)
dnomid_no_all_bigrams = dnomid_no_bigram['neighborhood_overview'].apply(get_bigrams).sum()


get_lda_topics(dnomid_no_all_bigrams)

[(0,
  '0.003*"tennessee_state" + 0.003*"marathon_village" + 0.002*"mins._tennessee" + 0.002*"mins._nashville" + 0.002*"walking_distance" + 0.002*"hall_fame" + 0.002*"music_row" + 0.002*"ole_opry" + 0.002*"east_nashville" + 0.002*"nashville\'s_best"'),
 (1,
  '0.001*"marathon_village" + 0.001*"music_row" + 0.001*"hall_fame" + 0.001*"tennessee_state" + 0.001*"music_city" + 0.001*"walking_distance" + 0.001*"country_music" + 0.001*"mins._nashville" + 0.001*"grand_ole" + 0.001*"nashville\'s_best"')]

###D&N_O TOP

In [ ]:
dnotop_d_bigram = dnotop.copy()
dnotop_d_bigram.dropna(subset=['description'], inplace=True)
dnotop_d_all_bigrams = dnotop_d_bigram['description'].apply(get_bigrams).sum()


get_lda_topics(dnotop_d_all_bigrams)

[(0,
  '0.002*"living_room" + 0.001*"walking_distance" + 0.001*"brand_new" + 0.001*"music_city" + 0.001*"east_nashville" + 0.001*"fully_equipped" + 0.001*"downtown_nashville" + 0.001*"fully_stocked" + 0.001*"queen_bed" + 0.001*"king_bed"'),
 (1,
  '0.000*"living_room" + 0.000*"music_city" + 0.000*"walking_distance" + 0.000*"brand_new" + 0.000*"east_nashville" + 0.000*"downtown_nashville" + 0.000*"fully_stocked" + 0.000*"queen_bed" + 0.000*"fully_equipped" + 0.000*"living_area"')]

In [ ]:
dnotop_no_bigram = dnotop.copy()
dnotop_no_bigram.dropna(subset=['neighborhood_overview'], inplace=True)
dnotop_no_all_bigrams = dnotop_no_bigram['neighborhood_overview'].apply(get_bigrams).sum()


get_lda_topics(dnotop_no_all_bigrams)

[(0,
  '0.000*"east_nashville" + 0.000*"walking_distance" + 0.000*"blocks_-" + 0.000*"country_music" + 0.000*"music_city" + 0.000*"music_hall" + 0.000*"downtown_nashville" + 0.000*"miles_-" + 0.000*"min_walk:" + 0.000*"within_walking"'),
 (1,
  '0.004*"east_nashville" + 0.003*"walking_distance" + 0.002*"blocks_-" + 0.002*"min_walk:" + 0.002*"country_music" + 0.002*"music_hall" + 0.001*"music_city" + 0.001*"within_walking" + 0.001*"downtown_nashville" + 0.001*"miles_-"')]

### Reviews

In [ ]:
revlow_bigram = revlow.copy()
revlow_bigram.dropna(inplace=True)
revlow_allbigrams = revlow_bigram['comments'].apply(get_bigrams).sum()
get_lda_topics(revlow_allbigrams)

[(0,
  '0.002*"air_bnb" + 0.002*"first_time" + 0.001*"day_trip" + 0.001*"code_garage" + 0.001*"st._thomas" + 0.001*"agreed_someone" + 0.001*"front_door" + 0.001*"–_tried" + 0.001*"clearly_cleaned" + 0.001*"stay_nashville."'),
 (1,
  '0.001*"first_time" + 0.001*"air_bnb" + 0.001*"steps_get" + 0.001*"host_day" + 0.001*"able_get" + 0.001*"bnb_said" + 0.001*"someone_came" + 0.001*"location_great" + 0.001*"garage_door" + 0.001*"5_stars."')]

In [ ]:
revmid_bigram = revmid.copy()
revmid_bigram.dropna(inplace=True)
revmid_allbigrams = revmid_bigram['comments'].apply(get_bigrams).sum()
get_lda_topics(revmid_allbigrams)

[(0,
  '0.000*"great_place" + 0.000*"would_definitely" + 0.000*"stay_again." + 0.000*"place_stay" + 0.000*"great_location" + 0.000*"living_room" + 0.000*"uber_ride" + 0.000*"nice_place" + 0.000*"great_stay" + 0.000*"walking_distance"'),
 (1,
  '0.001*"great_place" + 0.001*"would_definitely" + 0.001*"place_stay" + 0.001*"stay_again." + 0.001*"definitely_stay" + 0.001*"living_room" + 0.001*"would_stay" + 0.001*"uber_ride" + 0.001*"great_location" + 0.001*"close_downtown"')]

In [ ]:
revtop_bigram = revtop.copy()
revtop_bigram.dropna(inplace=True)
revtop_allbigrams = revtop_bigram['comments'].apply(get_bigrams).sum()
get_lda_topics(revtop_allbigrams)

### Creating DF


In [ ]:
words = []
for topic, weights in get_lda_topics(dnolow_d_all_bigrams):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))

df9 = pd.DataFrame(words, columns=['Low Rated Description Bigrams'])

for topic, weights in get_lda_topics(dnomid_d_all_bigrams):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))
df10 = pd.DataFrame(words, columns=['Mid Rated Description Bigrams'])

for topic, weights in get_lda_topics(dnotop_d_all_bigrams):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))
df11 = pd.DataFrame(words, columns=['Top Rated Description Bigrams'])


df_combined_dbigrams = pd.concat([df9, df10,df11], axis=1)
#df_combined_dbigrams.to_csv('DescriptionBigramsLDA',index=False)
df_combined_dbigrams

,Low Rated Description Bigrams,Mid Rated Description Bigrams,Top Rated Description Bigrams
0,1_queen,away_home,away_home
1,away_home,"bedroom,_1","bedroom,_1"
2,newly_renovated,king_bed,king_bed
3,music_city.,minutes_away,minutes_away
4,country_music,stainless_steel,queen_size
5,living_room,walking_track,east_nashville
6,"bedroom,_1","steel_appliances,",stainless_steel
7,1_king,newly_renovated,walking_track
8,king_bed,music_city.,"steel_appliances,"
9,stainless_steel,living_area,newly_renovated


In [ ]:
words = []
for topic, weights in get_lda_topics(dnolow_no_all_bigrams):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))

df12 = pd.DataFrame(words, columns=['Low Rated Neighborhood Overview Bigrams'])

for topic, weights in get_lda_topics(dnomid_no_all_bigrams):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))
df13 = pd.DataFrame(words, columns=['Mid Rated Neighborhood Overview Bigrams'])

for topic, weights in get_lda_topics(dnotop_no_all_bigrams):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))
df14 = pd.DataFrame(words, columns=['Top Rated Neighborhood Overview Bigrams'])


df_combined_nobigrams = pd.concat([df12, df13,df14], axis=1)
#df_combined_nobigrams.to_csv('NeighborhoodOverviewBigramsLDA',index=False)
df_combined_nobigrams

,Low Rated Neighborhood Overview Bigrams,Mid Rated Neighborhood Overview Bigrams,Top Rated Neighborhood Overview Bigrams
0,"(6.4_miles),",mins._tennessee,within_walking
1,mins._tennessee,park10_mins.,mins._tennessee
2,park10_mins.,tennessee_state,park10_mins.
3,capitol_mall,mall_state,mall_state
4,museum11_mins.,bicentennial_capitol,tennessee_state
5,music_row,"(6.4_miles),",bicentennial_capitol
6,tennessee_state,east_nashville,"(6.4_miles),"
7,hall_fame,music_row,east_nashville
8,mall_state,museum11_mins.,music_row
9,quiet_neighborhood,grand_ole,museum11_mins.


In [ ]:
words = []
for topic, weights in get_lda_topics(revlow_allbigrams):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))

df15 = pd.DataFrame(words, columns=['Low Review Bigrams'])

for topic, weights in get_lda_topics(revmid_allbigrams):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))
df16 = pd.DataFrame(words, columns=['Mid Review Bigrams'])

for topic, weights in get_lda_topics(revtop_allbigrams):
    words.extend(re.findall(r'"([^"]*)"', weights))
words = list(set(words))
df17 = pd.DataFrame(words, columns=['Top Review Bigrams'])


df_combined_rev = pd.concat([df15, df16,df17], axis=1)
#df_combined_rev.to_csv('ReviewBigramsLDA',index=False)
df_combined_rev

NameError: ignored

##Frequent Bigrams

In [ ]:
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

nltk.download('stopwords')

def extract_top_bigrams(df, col_name, n):

    text = ' '.join(df[col_name].astype(str).tolist())
    text = re.sub('[^A-Za-z0-9]+', ' ', text.lower())
    text = text.replace('br', '') # Remove "br"
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]

   
    bigram_measures = BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(words)
    top_bigrams = finder.nbest(bigram_measures.raw_freq, n)

    
    bigram_freqs = finder.ngram_fd.items()

    
    top_bigram_freqs = [freq for freq in bigram_freqs if freq[0] in top_bigrams]

    return top_bigram_freqs

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
extract_top_bigrams(revlow,'comments',10)

[(('first', 'time'), 4),
 (('5', 'stars'), 2),
 (('stay', 'nashville'), 3),
 (('air', 'bnb'), 4),
 (('washer', 'dryer'), 3),
 (('living', 'room'), 4),
 (('access', 'codes'), 2),
 (('able', 'get'), 2),
 (('garage', 'door'), 3),
 (('1', 'bedroom'), 2)]

In [ ]:
extract_top_bigrams(revmid,'comments',10)

[(('place', 'clean'), 29),
 (('great', 'location'), 51),
 (('living', 'room'), 29),
 (('great', 'place'), 59),
 (('great', 'stay'), 37),
 (('place', 'stay'), 49),
 (('nice', 'place'), 25),
 (('would', 'definitely'), 33),
 (('definitely', 'stay'), 22),
 (('close', 'downtown'), 35)]

In [ ]:
extract_top_bigrams(revtop,'comments',10)

[(('would', 'definitely'), 42914),
 (('place', 'stay'), 43578),
 (('place', 'great'), 21834),
 (('definitely', 'stay'), 32573),
 (('stay', 'great'), 21145),
 (('great', 'location'), 66215),
 (('great', 'stay'), 19976),
 (('highly', 'recommend'), 38965),
 (('great', 'place'), 45497),
 (('walking', 'distance'), 23979)]

In [ ]:
words1 = [word for word, _ in extract_top_bigrams(revlow,'comments',10)]
words2 = [word for word, _ in extract_top_bigrams(revmid,'comments',10)]
words3 = [word for word, _ in extract_top_bigrams(revtop,'comments',10)]

df = pd.DataFrame({'Low Review Frequent Bigrams': words1, 'Mid Review Frequent Bigrams': words2, 'Top Review Frequent Bigrams': words3})
#df.to_csv('Review_Frequent_Bigrams',index=False)
df

,Low Review Frequent Bigrams,Mid Review Frequent Bigrams,Top Review Frequent Bigrams
0,"(first, time)","(place, clean)","(would, definitely)"
1,"(5, stars)","(great, location)","(place, stay)"
2,"(stay, nashville)","(living, room)","(place, great)"
3,"(air, bnb)","(great, place)","(definitely, stay)"
4,"(washer, dryer)","(great, stay)","(stay, great)"
5,"(living, room)","(place, stay)","(great, location)"
6,"(access, codes)","(nice, place)","(great, stay)"
7,"(able, get)","(would, definitely)","(highly, recommend)"
8,"(garage, door)","(definitely, stay)","(great, place)"
9,"(1, bedroom)","(close, downtown)","(walking, distance)"


##Frequent Words

In [ ]:
from collections import Counter

def extract_top_words(df, column_name, num_words):

    all_words = []
    stop_words = set(stopwords.words('english'))

    for index, row in df.iterrows():
        text = row[column_name]
   
        if isinstance(text, str):
            text = text.lower()
            text = text.translate(str.maketrans('', '', string.punctuation))
            words = text.split()
            words = [word for word in words if word not in stop_words]

            all_words.extend(words)

    word_counts = Counter(all_words)

    sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    return sorted_words[:num_words]

In [ ]:
revlow_words = extract_top_words(revlow,'comments',20)
revlow_words

[('stay', 20),
 ('host', 17),
 ('get', 16),
 ('home', 15),
 ('airbnb', 15),
 ('place', 14),
 ('hotel', 14),
 ('would', 11),
 ('one', 11),
 ('–', 11),
 ('room', 10),
 ('house', 10),
 ('bed', 10),
 ('work', 10),
 ('could', 9),
 ('night', 9),
 ('time', 9),
 ('unit', 9),
 ('bedroom', 8),
 ('air', 8)]

In [ ]:
revmid_words = extract_top_words(revmid,'comments',20)
revmid_words

[('place', 354),
 ('stay', 349),
 ('great', 325),
 ('location', 214),
 ('clean', 212),
 ('would', 210),
 ('nice', 165),
 ('room', 155),
 ('get', 150),
 ('us', 130),
 ('nashville', 124),
 ('downtown', 119),
 ('one', 119),
 ('good', 112),
 ('unit', 110),
 ('also', 105),
 ('close', 103),
 ('night', 103),
 ('area', 103),
 ('host', 103)]

In [ ]:
revtop_words = extract_top_words(revtop,'comments',20)
revtop_words

[('great', 337421),
 ('stay', 261885),
 ('place', 258621),
 ('location', 181040),
 ('nashville', 161281),
 ('clean', 154675),
 ('would', 129564),
 ('house', 103962),
 ('perfect', 103159),
 ('everything', 100693),
 ('downtown', 94425),
 ('recommend', 87905),
 ('definitely', 87631),
 ('nice', 84065),
 ('home', 83284),
 ('comfortable', 83028),
 ('close', 76683),
 ('host', 75248),
 ('easy', 71785),
 ('us', 71160)]

In [ ]:

words1 = [word for word, _ in revlow_words]
words2 = [word for word, _ in revmid_words]
words3 = [word for word, _ in revtop_words]

df = pd.DataFrame({'Low Review Frequent Words': words1, 'Mid Review Frequent Words': words2, 'Top Review Frequent Words': words3})
#df.to_csv('Review_Frequent_Words',index=False)
df

In [ ]:
dnolow_words = extract_top_words(dnolow,'description',20)
dnolow_words

[('br', 95),
 ('home', 26),
 ('•', 26),
 ('nashville', 24),
 ('room', 22),
 ('music', 22),
 ('bedroom', 20),
 ('kitchen', 20),
 ('1', 19),
 ('bthe', 18),
 ('spacebbr', 18),
 ('downtown', 18),
 ('living', 15),
 ('2', 14),
 ('area', 13),
 ('full', 13),
 ('bed', 13),
 ('minutes', 12),
 ('guests', 12),
 ('private', 12)]

In [ ]:
dnomid_words = extract_top_words(dnomid,'description',20)
dnomid_words

[('br', 456),
 ('nashville', 120),
 ('bthe', 91),
 ('spacebbr', 91),
 ('downtown', 86),
 ('•', 84),
 ('kitchen', 84),
 ('living', 82),
 ('home', 81),
 ('bedroom', 72),
 ('room', 65),
 ('space', 64),
 ('area', 59),
 ('music', 57),
 ('minutes', 56),
 ('2', 50),
 ('located', 48),
 ('stay', 47),
 ('private', 46),
 ('enjoy', 45)]

In [ ]:
dnotop_words = extract_top_words(dnotop,'description',20)
dnotop_words

[('br', 27023),
 ('nashville', 8375),
 ('home', 7179),
 ('bthe', 5682),
 ('spacebbr', 5678),
 ('downtown', 5003),
 ('room', 4922),
 ('bedroom', 4764),
 ('kitchen', 4484),
 ('2', 4470),
 ('living', 3909),
 ('space', 3715),
 ('bed', 3686),
 ('full', 3664),
 ('private', 3613),
 ('music', 3558),
 ('located', 3414),
 ('1', 3213),
 ('queen', 2978),
 ('enjoy', 2787)]

# NMF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

nmfdf['description'] = dnolow['description'].fillna('')

tfidf_vectorizer = TfidfVectorizer(max_features=1500, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(nmfdf['description'])
words = np.array(tfidf_vectorizer.get_feature_names_out())

nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(tfidf)
H = nmf.components_

topics_df = pd.DataFrame(columns=[f"Topic {i+1}" for i in range(nmf.n_components)])
for i, topic in enumerate(H):
    top_words = [words[j] for j in topic.argsort()[:-11:-1]]
    topics_df[f"Topic {i+1}"] = top_words

topics_df.to_csv('DescriptionLowNMF',index=False)


In [ ]:
nmfdf['description'] = dnomid['description'].fillna('')

tfidf_vectorizer = TfidfVectorizer(max_features=1500, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(nmfdf['description'])
words = np.array(tfidf_vectorizer.get_feature_names_out())

nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(tfidf)
H = nmf.components_

topics_df = pd.DataFrame(columns=[f"Topic {i+1}" for i in range(nmf.n_components)])
for i, topic in enumerate(H):
    top_words = [words[j] for j in topic.argsort()[:-11:-1]]
    topics_df[f"Topic {i+1}"] = top_words

topics_df.to_csv('DescriptionMidNMF',index=False)

In [ ]:
nmfdf['description'] = dnotop['description'].fillna('')

tfidf_vectorizer = TfidfVectorizer(max_features=1500, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(nmfdf['description'])
words = np.array(tfidf_vectorizer.get_feature_names_out())

nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(tfidf)
H = nmf.components_

topics_df = pd.DataFrame(columns=[f"Topic {i+1}" for i in range(nmf.n_components)])
for i, topic in enumerate(H):
    top_words = [words[j] for j in topic.argsort()[:-11:-1]]
    topics_df[f"Topic {i+1}"] = top_words

topics_df.to_csv('DescriptionTopNMF',index=False)

In [ ]:
nmfdf['comments'] = revlow['comments'].fillna('')

tfidf_vectorizer = TfidfVectorizer(max_features=1500, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(nmfdf['comments'])
words = np.array(tfidf_vectorizer.get_feature_names_out())

nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(tfidf)
H = nmf.components_

topics_df = pd.DataFrame(columns=[f"Topic {i+1}" for i in range(nmf.n_components)])
for i, topic in enumerate(H):
    top_words = [words[j] for j in topic.argsort()[:-11:-1]]
    topics_df[f"Topic {i+1}"] = top_words

topics_df.to_csv('ReviewsLowNMF',index=False)

In [ ]:
nmfdf['comments'] = revmid['comments'].fillna('')

tfidf_vectorizer = TfidfVectorizer(max_features=1500, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(nmfdf['comments'])
words = np.array(tfidf_vectorizer.get_feature_names_out())

nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(tfidf)
H = nmf.components_

topics_df = pd.DataFrame(columns=[f"Topic {i+1}" for i in range(nmf.n_components)])
for i, topic in enumerate(H):
    top_words = [words[j] for j in topic.argsort()[:-11:-1]]
    topics_df[f"Topic {i+1}"] = top_words

topics_df.to_csv('ReviewsMidNMF',index=False)








In [ ]:
nmfdf['comments'] = revtop['comments'].fillna('')

tfidf_vectorizer = TfidfVectorizer(max_features=1500, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(nmfdf['comments'])
words = np.array(tfidf_vectorizer.get_feature_names_out())

nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(tfidf)
H = nmf.components_

topics_df = pd.DataFrame(columns=[f"Topic {i+1}" for i in range(nmf.n_components)])
for i, topic in enumerate(H):
    top_words = [words[j] for j in topic.argsort()[:-11:-1]]
    topics_df[f"Topic {i+1}"] = top_words

topics_df.to_csv('ReviewsTopNMF',index=False)

In [ ]:

nmfdf['neighborhood_overview'] = dnolow['neighborhood_overview'].fillna('')

tfidf_vectorizer = TfidfVectorizer(max_features=1500, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(nmfdf['neighborhood_overview'])
words = np.array(tfidf_vectorizer.get_feature_names_out())

nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(tfidf)
H = nmf.components_

topics_df = pd.DataFrame(columns=[f"Topic {i+1}" for i in range(nmf.n_components)])
for i, topic in enumerate(H):
    top_words = [words[j] for j in topic.argsort()[:-11:-1]]
    topics_df[f"Topic {i+1}"] = top_words

topics_df.to_csv('NOLowNMF',index=False)

/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
nmfdf['neighborhood_overview'] = dnomid['neighborhood_overview'].fillna('')

tfidf_vectorizer = TfidfVectorizer(max_features=1500, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(nmfdf['neighborhood_overview'])
words = np.array(tfidf_vectorizer.get_feature_names_out())

nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(tfidf)
H = nmf.components_

topics_df = pd.DataFrame(columns=[f"Topic {i+1}" for i in range(nmf.n_components)])
for i, topic in enumerate(H):
    top_words = [words[j] for j in topic.argsort()[:-11:-1]]
    topics_df[f"Topic {i+1}"] = top_words

topics_df.to_csv('NOMidNMF',index=False)

/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
nmfdf['neighborhood_overview'] = dnotop['neighborhood_overview'].fillna('')

tfidf_vectorizer = TfidfVectorizer(max_features=1500, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(nmfdf['neighborhood_overview'])
words = np.array(tfidf_vectorizer.get_feature_names_out())

nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(tfidf)
H = nmf.components_

topics_df = pd.DataFrame(columns=[f"Topic {i+1}" for i in range(nmf.n_components)])
for i, topic in enumerate(H):
    top_words = [words[j] for j in topic.argsort()[:-11:-1]]
    topics_df[f"Topic {i+1}"] = top_words

topics_df.to_csv('NOTopNMF',index=False)

# Sentiment